Anomaly Detection in Time Series Data

Importing Libraries and Dataset

In [1]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [7]:
import sys
print(sys.version)


3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]


In [9]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)


TensorFlow Version: 2.18.0


In [11]:
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt

Dataset Load

In [13]:
data = pd.read_csv(r"C:\Users\rajni\Downloads\ambient_temperature_system_failure.csv"
    )

# Exclude datetime column
data_values = data.drop('timestamp',
                        axis=1).values

# Convert data to float type
data_values = data_values.astype('float32')

# Create new dataframe with converted values
data_converted = pd.DataFrame(data_values,
                              columns=data.columns[1:])

# Add back datetime column
data_converted.insert(0, 'timestamp',
                      data['timestamp'])

We load a dataset called “ambient_temperature_system_failure.csv” from the Numenta Anomaly Benchmark (NAB) dataset, which contains time-series data of ambient temperature readings from a system that experienced a failure. 

The panda’s library is used to read the CSV file from a remote location on GitHub and store it in a variable called “data”.

Now, the code drops the “timestamp” column from the “data” variable, since it is not needed for data analysis purposes. The remaining columns are stored in a variable called “data_values”.
Then, the “data_values” are converted to the “float32” data type to reduce memory usage, and a new pandas DataFrame called “data_converted” is created with the converted data. The columns of “data_converted” are labeled with the original column names from “data”, except for the “timestamp” column that was previously dropped.
Finally, the code adds the “timestamp” column back to “data_converted” at the beginning using the “insert()” method. The resulting DataFrame “data_converted” has the same data as “data” but without the unnecessary “timestamp” column, and the data is in a format that can be used for analysis and visualization.

In [15]:
data_converted = data_converted.dropna()

We remove any missing or NaN values from the dataset.

Anomaly Detection using Autoencoder

In [17]:
# Exclude datetime column again
data_tensor = tf.convert_to_tensor(data_converted.drop(
    'timestamp', axis=1).values, dtype=tf.float32)

# Define the autoencoder model
input_dim = data_converted.shape[1] - 1
encoding_dim = 10

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='relu')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compile and fit the model
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(data_tensor, data_tensor, epochs=50,
                batch_size=32, shuffle=True)

# Calculate the reconstruction error for each data point
reconstructions = autoencoder.predict(data_tensor)
mse = tf.reduce_mean(tf.square(data_tensor - reconstructions),
                     axis=1)
anomaly_scores = pd.Series(mse.numpy(), name='anomaly_scores')
anomaly_scores.index = data_converted.index

Epoch 1/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 960us/step - loss: 5101.3555
Epoch 2/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5092.4229
Epoch 3/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step - loss: 5084.2041
Epoch 4/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 816us/step - loss: 5086.8784
Epoch 5/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - loss: 5098.0425
Epoch 6/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 890us/step - loss: 5103.9961
Epoch 7/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - loss: 5099.9780
Epoch 8/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - loss: 5101.6445
Epoch 9/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 876us/step - loss: 5096.1777
Epoch 10/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - loss: 5088.2158
Epoch 11/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 5092.4517
Epoch 12/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - loss: 5081.6851
Epoch 13/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - loss: 5101.7231
Epoch 14/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - loss:

We define the autoencoder model and fit it to the cleaned data. The autoencoder is used to identify any deviations from the regular patterns in the data that are learned from the data. To reduce the mean squared error between the input and the output, the model is trained. The reconstruction error for each data point is determined using the trained model and is utilized as an anomaly score.

In [19]:
threshold = anomaly_scores.quantile(0.99)
anomalous = anomaly_scores > threshold
binary_labels = anomalous.astype(int)
precision, recall,\
    f1_score, _ = precision_recall_fscore_support(
        binary_labels, anomalous, average='binary')

Here, we define an anomaly detection threshold and assess the model’s effectiveness using precision, recall, and F1 score. Recall is the ratio of true positives to all real positives, whereas precision is the ratio of genuine positives to all projected positives. The harmonic mean of recall and accuracy is the F1 score.


In [21]:
test = data_converted['value'].values
predictions = anomaly_scores.values

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1_score)

Precision:  1.0
Recall:  1.0
F1 Score:  1.0


Visualizing the Anomaly

In [ ]:
# Plot the data with anomalies marked in red
plt.figure(figsize=(16, 8))
plt.plot(data_converted['timestamp'],
         data_converted['value'])
plt.plot(data_converted['timestamp'][anomalous],
         data_converted['value'][anomalous], 'ro')
plt.title('Anomaly Detection')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()